In [37]:
import os
from dotenv import load_dotenv
import requests
import time
import json

In [53]:
load_dotenv()
city_name = "Bangkok"
openweathermap_api_endpoint ="https://api.openweathermap.org/data/2.5/weather?q="+city_name+"&appid="+os.getenv("API_KEY")+"&units=metric&lang=en"

In [54]:
response = requests.get(openweathermap_api_endpoint) #fetch API
response.json()

{'coord': {'lon': 100.5167, 'lat': 13.75},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 26.76,
  'feels_like': 26.56,
  'temp_min': 25.94,
  'temp_max': 29.95,
  'pressure': 1014,
  'humidity': 37,
  'sea_level': 1014,
  'grnd_level': 1013},
 'visibility': 10000,
 'wind': {'speed': 2.22, 'deg': 7, 'gust': 3.36},
 'clouds': {'all': 90},
 'dt': 1703226804,
 'sys': {'type': 1,
  'id': 9235,
  'country': 'TH',
  'sunrise': 1703201805,
  'sunset': 1703242531},
 'timezone': 25200,
 'id': 1609350,
 'name': 'Bangkok',
 'cod': 200}

In [78]:
weather=response.json()
temp = weather["main"]["temp"]
humidity = weather["main"]["humidity"]
city = weather['name']
x = weather['weather'][0]
x

{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}

In [57]:
json_message = {"City":city, "Tempurature": temp, "Humidity": humidity, "CreationTime": time.strftime("%Y-%m-%d %H:%M:%S")}
print(json.dumps(json_message))

{"City": "Bangkok", "Tempurature": 26.76, "Humidity": 37, "CreationTime": "2023-12-22 13:44:33"}


In [69]:
openweathermap_api_endpoint ="https://api.openweathermap.org/data/2.5/weather?q="+city_name+"&appid="+os.getenv("API_KEY")+"&units=metric&lang=en"

In [70]:
def get_data(city_name):
    import requests
    import os
    from dotenv import load_dotenv
    load_dotenv()
    openweathermap_api_endpoint ="https://api.openweathermap.org/data/2.5/weather?q="+city_name+"&appid="+os.getenv("API_KEY")+"&units=metric&lang=en"
    res = requests.get(openweathermap_api_endpoint)
    res = res.json()

    return res

In [79]:

res1 = get_data(city_name = "Bangkok")
print(res1['name'])
print(res1['weather'][0])

res2 = get_data(city_name = "London")
print(res2['name'])
print(res2['weather'][0])

Bangkok
{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}
London
{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pandas as pd
import json

In [13]:
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.13:3.4.1,com.datastax.spark:spark-cassandra-connector_2.13:3.4.1").\
config('spark.cassandra.connection.host', 'localhost'). \
getOrCreate()

In [14]:
data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "weather_monitoring")\
  .option("startingOffsets", "earliest")\
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").alias("data").select("data.value")

In [5]:
data.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
schema = StructType([
        StructField("CityName", StringType()),
        StructField("Temperature", DoubleType()),
        StructField("Humidity", DoubleType()),
        StructField("Weather", StringType()),
        StructField("CreationTime", StringType()),
    ])

In [16]:
sel = data.selectExpr("CAST(value AS STRING)") \
        .select(from_json(col('value'), schema).alias('data')).select("data.*")
print(sel)

DataFrame[CityName: string, Temperature: double, Humidity: double, Weather: string, CreationTime: string]
